In [2]:
import collections
import math
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import nltk

In [18]:
ppp = nltk.data.load('../Downloads/ppp.txt', encoding='utf8')
words_p = nltk.tokenize.word_tokenize(ppp)[122:]    

In [19]:
alw = nltk.data.load('../Downloads/alw.txt', encoding='utf8')
words_a = nltk.tokenize.word_tokenize(alw)[122:]

In [20]:
words = words_a + words_p

In [21]:
l2 = np.load('./l2_embed3.npy')
centroids = np.load('./centroidsl2.npy').item()
word_embed = np.load('./embed3.npy')
centers = centroids.cluster_centers_
c_labels = centroids.labels_ + len(word_embed)
c_counts = centroids.counts_

In [22]:
vocabulary_size = 10000
def build_dataset(words):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
del words_a
del words_p # Hint to reduce memory.
print('Most common words (+UNK)', count[:20])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0

Most common words (+UNK) [['UNK', 40], (',', 11834), ('the', 5884), ('.', 5086), ('to', 4942), ('of', 4298), ('and', 4280), ('a', 2623), ('her', 2349), ('in', 2239), ('was', 2186), ('I', 2040), ('she', 1870), ('it', 1737), ('that', 1733), (';', 1732), ('not', 1667), ('you', 1504), ('be', 1414), ('as', 1375)]
Sample data [1334, 11, 3, 3771, 2, 7286, 65, 10, 659, 4] ['CHAPTER', 'I', '.', 'Down', 'the', 'Rabbit-Hole', 'Alice', 'was', 'beginning', 'to']


In [23]:
comb_embed = np.concatenate((word_embed, centers), axis=0)

In [24]:
transition_dict = {}
for index, transition in enumerate(l2):
    f  = transition[-2].astype(int)
    t = transition[-1].astype(int)
    if not (f,t) in transition_dict and c_counts[c_labels[index] - len(word_embed)] > 10:
        transition_dict[tuple([f,t])] = c_labels[index]
        dictionary['l2_' + str(c_labels[index])] = c_labels[index]
        reverse_dictionary[c_labels[index]] = 'l2_' + str(c_labels[index])


In [25]:
for index, label in enumerate(data[:-1]):
    if (label, data[index+1]) in transition_dict:
        data.insert(index+1, transition_dict[(label, data[index+1])])

In [26]:
del words

In [27]:
vocabulary_size += len(centers)

In [233]:
sentences_vec = []
sentences_word  = []
sent_word = []
sent_vec = []
indices = []
for index, word in enumerate(filter(lambda word: word in dictionary, words)):
    if word in ['?','.','!']:
        sent_vec += [comb_embed[dictionary[word]]]
        sent_word += [word]
        if len(sent_vec) > 2:
            sn_w = [sent_word[0]]
            sn_v = [sent_vec[0]]
            for index, word in enumerate(sent_word[:-1]):
                if tuple([word, sent_word[index+1]]) in transition_dict:
                    pair = tuple([word, sent_word[index+1]])
                    sn_v += [comb_embed[transition_dict[pair]], sent_vec[index+1]]
                    sn_w += ['l2_'+str(transition_dict[pair]), sent_word[index+1]]
            sentences_vec += [sn_v]
            sentences_word += [sn_w]
            sn_v = []
            sn_w = []
            sent_vec = []
            sent_word = []
    elif word in ['Chapter', 'CHAPTER'] and words[index+1] in ['1','I']:
        indices += [len(sentences_vec)]
    else:
        sent_vec += [comb_embed[dictionary[word]]]
        sent_word += [word]

NameError: name 'words' is not defined

In [28]:

# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1 # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [ skip_window ]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=3)
for i in range(8):
  print(batch[i], reverse_dictionary[batch[i]],
      '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

# Step 4: Build and train a skip-gram model.

batch_size = 200
embedding_size = 100  # Dimension of the embedding vector.
skip_window = 6       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 50    # Random set of words to evaluate similarity on.
valid_window = 200  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 70    # Number of negative examples to sample.

3 . -> 1334 CHAPTER
3 . -> 3771 Down
3771 Down -> 65 Alice
3771 Down -> 3 .
2 the -> 11106 l2_11106
2 the -> 7286 Rabbit-Hole
7286 Rabbit-Hole -> 3771 Down
7286 Rabbit-Hole -> 10 was


In [29]:
graph = tf.Graph()

with graph.as_default():

  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
        tf.constant(comb_embed.astype(np.float32), shape=[vocabulary_size, embedding_size]))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  loss = tf.reduce_mean(
      tf.nn.nce_loss(nce_weights, nce_biases, embed, train_labels,
                     num_sampled, vocabulary_size))

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

  # Add variable initializer.
  init = tf.initialize_all_variables()

# Step 5: Begin training.
num_steps = 30001

with tf.Session(graph=graph) as session:
    # We must initialize all variables before we use them.
    init.run()
    print("Initialized")
    
    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(
            batch_size, num_skips, skip_window)
        feed_dict = {train_inputs : batch_inputs, train_labels : batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val
        
        
        if step % 2000 == 0:
          if step > 0:
            average_loss /= 2000
          # The average loss is an estimate of the loss over the last 2000 batches.
          print("Average loss at step ", step, ": ", average_loss)
          average_loss = 0
    
    
    final_embeddings = normalized_embeddings.eval()
    

Initialized
Average loss at step  0 :  233.568862915
Average loss at step  2000 :  44.6333351347
Average loss at step  4000 :  8.1805458833
Average loss at step  6000 :  5.29029760456
Average loss at step  8000 :  4.74318932462
Average loss at step  10000 :  4.5924071368
Average loss at step  12000 :  4.58445866847
Average loss at step  14000 :  4.56426617825
Average loss at step  16000 :  4.55514658105
Average loss at step  18000 :  4.51609221542
Average loss at step  20000 :  4.5192394985
Average loss at step  22000 :  4.51931215131
Average loss at step  24000 :  4.46344120216
Average loss at step  26000 :  4.46796448791
Average loss at step  28000 :  4.48009866548
Average loss at step  30000 :  4.44888775063


In [237]:
import seaborn as sn
import matplotlib.pyplot as plt 
% matplotlib inline

In [243]:
def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
    assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
    plt.figure(figsize=(18, 18))  #in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i,:]
        plt.scatter(x, y)
        plt.annotate(label,
                     xy=(x, y),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')

    plt.savefig(filename)

try:
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt

    tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
    plot_only = np.random.choice(np.arange(len(final_embeddings)), replace=False, size=500)
    low_dim_embs = tsne.fit_transform(plot_only)
    labels = [reverse_dictionary[i] for i in xrange(500)]
    plot_with_labels(low_dim_embs, labels)
except ImportError:
  print("Please install sklearn, matplotlib, and scipy to visualize embeddings.")


/home/lenny/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: total size of new array must be unchanged

In [30]:
np.save('./l2embed', final_embeddings)

## The Plan

### Step 1
Train some actual word2vec sets. Understand how to use it. Then use CNNs for Language Understanding Stuff.
Use Gutenberg Library. Don't even think about changing the functions yet. Just use the for now.

### Step 2
Work through some analogical reasoning stuff. Practice optimizing, but ONLY as a means of understanding how to work with stuff. Try to understand how you would do author recognition with CNNs.

### Step 3
Read more papers. Understand more about what you've actually been doing. Write up some stuff on the link between CNNs and what you're doing. 

### Step 4
Implement a version of word2vec that allows for an increase in the vocabulary. 

### Step 5a
Implement a version of word2vec that adds bigram transitions. The main focus here is clustering transitions and integration of unknown vocab. When we see a known transition, we add it into the context.

### Step 5b
Add a graph on top of the network and begin looking at methods to choose link likelihood

### Step 6
Combine network and word2vec with bigram transitions. Recognize words as transitions. Especially that. Actually. You might even just try recognizing 3 word transitions like $a \xrightarrow{b} c$, that's probably the most concrete place to start. Also play around with a high dropout rate. So train batch. Drop / cluster a bunch then continue.

### Step 7
Try implimenting adding in trigrams inbetween bigrams. Basically, train two contexts once you've trained bigrams.
Also, implement adding of links for likely things. We are trying to optimize the likelihood, given some word, of the links from that word. 

### Step 7b
This is never going to work

### Step 8
Try to forget the hierarchical stuff. Try seeing how high features can get. Assume a fixed vocabulary size, but words not in the vocab can be added as higher level relations. Actually, try to merge transitions with words. Fixed graph size $|V|^2$

### Step 9a
Try to implement some fancy stability constraint stuff

### Step 9b
Try to get this system to recognize authors. You can train a classical neural network, but try it at different levels. Try to recognize authors using only the transitions between words. Even if this just performs decent, what does that mean? What interpretation could one draw. Perhaps this is closer to the idealized style recognition that has been hoped for in the past. Also, we can dry to reduce this to the minimum need transitions or maybe try to find singular identifying transitions per author.

### Step 10
Try to forcast the next sentence from the previous, or the next paragraph from the previous, etc. Try speech generation. Try to untangle mixed up words.